**Pré-Processamento - Mineração de Dados**


**Nome: Davi Augusto Neves Leite**

**Data de Entrega: 19/09/2023**


---


# <strong> Materiais </strong>


Os principais recursos para a execução desta atividade podem ser vistos a seguir.

1. <strong>Software</strong>

- Sistemas Operacionais: Windows 11 para _desktop_;
- Ambiente de Desenvolvimento Integrado: Microsoft Visual Studio Code;
- Linguagem de Programação: Python 3.11.5 64-bit.

2. <strong>Hardware</strong>

- Notebook pessoal Lenovo Ideapad 330-15IKB com: processador Intel Core i7-8550U, HDD WD Blue WD10SPZX de 1TB, SSD Crucial BX500 de 240GB, 12 GB DDR4 de Memória RAM e placa de vídeo NVIDIA GeForce MX150 (2 GB GDDR5 de memória).


---


# <strong> Importação das Bibliotecas Principais </strong>

Nota: ao decorrer deste Notebook, outras bibliotecas podem ser utilizadas em quaisquer respectiva seção/conjunto de dados, dependendo da necessidade. Abaixo, há a importação das principais que são comuns e utilizadas em todas ou quase todas seções/conjunto de dados.


In [ ]:
import numpy as np      # Manipulação de listas
import pandas as pd     # Manipulação de tabelas
import seaborn as sbn   # Geração de gráficos estatísticos
import matplotlib.pyplot as plt  # Geração de gráficos de listas
import sklearn as skl   # Biblioteca para pré-processamento
from copy import copy as cp  # Possibilitar copiar os objetos

# Ignorar os avisos não importantes durante a execução deste notebook
import warnings
warnings.filterwarnings('ignore')


---


# Conjunto Numérico: **_Rice (Cammeo and Osmancik)_**


**Descrição do Dataset:** este conjunto é composto por 3810 dados obtidos acerca de duas espécies diferentes de grãos de arroz na Turquia. Os dados estão compostos por 7 características morfológicas (atributos) destes grãos: área, perímetro, comprimento do eixo principal, comprimento do eixo menor, excentricidade, área convexa e extensão.

A descrição de cada atributo pode ser vista a seguir.

1. **Área (decimal):** número de pixels dentro dos limites do grão de arroz;
2. **Perímetro (decimal):** circunferência do grão de arroz por meio do cálculo da distância de pixels ao redor dos limites do grão de arroz;
3. **Comprimento do Eixo Principal (decimal):** linha mais longa que pode ser desenhada no grão de arroz;
4. **Comprimento do Eixo Menor (decimal):** linha mais curta que pode ser desenhada no grão de arroz;
5. **Excentricidade (decimal):** medida que diz respeito ao quão redonda é a elipse do grão de arroz;
6. **Área convexa (inteiro):** contagem de pixels da menor concha convexa da região formada pelo grão de arroz;
7. **Extensão (decimal):** proporção da região formada pelo grão de arroz em relação aos pixels da caixa delimitadora.

Especificamente, as espécies estudadas foram a _Osmancik_ e a _Cammeo_, ambas com características semelhantes de uma aparência larga, longa e sem brilho.

Este conjunto de dados pode ser acessado por meio de: [Rice (Cammeo and Osmancik)](https://archive.ics.uci.edu/dataset/545/rice+cammeo+and+osmancik)
(última data de acesso: 15 de set. de 2023).


## <strong> Informações Básicas </strong>


In [ ]:
# Acesso dos dados do dataset "Rice"
from scipy.io.arff import loadarff  # Carregar arquivo tipo .arff

dataset_rice_arff = loadarff('./Datasets/01_Rice_Cammeo_Osmancik.arff')
data = pd.DataFrame(data=dataset_rice_arff[0])

# Mostra os 5 primeiros e últimos registros
data


In [ ]:
# Mostra os 5 primeiros registros, formatados
data.head()


In [ ]:
# Mostra os 5 últimos registros, formatados
data.tail()


In [ ]:
# Mostra a quantidade de linhas e colunas da tabela (tupla)
data.shape


In [ ]:
# Mostra as informações dos atributos e outras do dataset
data.info()


## <strong> Exploração dos Dados </strong>


### Dados Simples: Média, Desvio-Padrão, Mínimo, Mediana, Máximo


In [ ]:
import pandas.api.types as pd_types  # Identificar o tipo de dado do dataset

# Percorrer cada atributo (coluna) e mostrar os dados estatísticos básicos de cada um
for col in data.columns:
    if pd_types.is_numeric_dtype(data[col]):
        print(f'{col}')
        print('\t Média = {:.2f}'.format(data[col].mean()))
        print('\t Desvio-Padrão = {:.2f}'.format(data[col].std()))
        print('\t Mínimo = {:.2f}'.format(data[col].min()))
        print('\t Mediana = {:.2f}'.format(data[col].median()))
        print('\t Máximo = {:.2f}'.format(data[col].max()))


### Quantidade de Dados de Cada Classe


In [ ]:
# Retornar a quantidade de classes do dataset por meio da coluna "Class"
# Nota: o nome da coluna deve ser exatamente igual ao do dataset (case-sensitive)
data['Class'].value_counts()


### Dados Estatísticos Completos Para Cada Atributo


Neste primeiro caso, são incluídos tanto os dados estatísticos básicos, como média e desvio padrão, quanto alguns dos mais avançados, como os percentis (25%, 50% e 75%). Vale ressaltar que neste caso há a análise para cada atributo separadamente, ou seja, dados como covariância, a qual relaciona os atributos entre si, não são mostrados.


In [ ]:
# Retornar, para cada atributo (coluna), a descrição estatística completa
# Incluem: média, frequência, mínimo, percentis (25, 50 e 75), dentre outros
data.describe(include='all')


É possível inferir, por exemplo, que a classe "Osmancik" é a mais recorrente com a existência de 2180 registros do total de 3810 deste _dataset_.


No caso abaixo, há a medida de correlação de cada par de atributos por meio do cálculo da chamada variância. A variança mede o quanto os dados estão dispersos em torno da média e, para isso, utiliza-se diretamente do desvio-padrão. Em termos práticos: quanto menor é a variância, mais próximos os valores estão da média.


In [ ]:
print('Covariância:')

# Mostrando os dados na forma de tabela
data.cov(numeric_only=True)


In [ ]:
# Mostrando na forma de mapa de calor
sbn.heatmap(data.cov(numeric_only=True), annot=True, cmap='YlGnBu')


Também, é possível visualizar a seguir os _boxplots_ de cada atributo, os quais mostram a distribuição de valores a partir dos limitantes inferior e superior e com uma "caixa" que indica a concentração de valores.


In [ ]:
# Plotando todos os boxplots num mesmo gráfico
data.boxplot(figsize=(12, 15))


Tomando como exemplo o atributo _Area_, é possível visualizar no gráfico acima que a maior concentração de dados está na faixa de valores do intervalo de _10000 e 15000_, com o mínimo sendo em _7500_ e o máximo sendo próximo de _20000_, sendo este máximo considerado um ruído (ou _outlier_, a ser visto na próxima seção).
Contudo, especialmente ao comparar os atributos de _Area_ e _Convex_Area_ com os demais nota-se que os dados plotados estão em escalas bem diferentes e, portanto, isso indica que deve ser realizado uma normalização deste conjunto para que seja possível analisar e, posteriormente, processar os dados de forma mais otimizada. A normalização será tratada na seção de **Pré-Processamento dos Dados**.


Por fim, outro tipo de gráfico bastante usado para análise é o de _scatter_. Abaixo, é possível visualizar os dados de cada classe como pontos na tupla de atributos relacionados _Area, Convex_Area_.


In [ ]:
sbn.scatterplot(data=data, x='Area', y='Convex_Area', hue='Class')


É possível inferir, por exemplo, que uma grande parte de dados da classe _Osmancik_ possui áreas que a classe _Cammeo_ não possui, nos intervalos de valores entre _0 à 10000_. Também, é possível visualizar que muitos dados das duas classes estão sobrepostos uns com os outros.


## <strong> Pré-Processamento dos Dados </strong>


O pré-processamento consiste na aplicação de diversas técnicas para limpar, selecionar e transformar os dados para melhorar a análise dos mesmos. Algumas técnicas: Agregação, _Sampling_, **_Feature Selection_**, **Redução da Dimensionalidade**, **_Feature Creation_**, Discretização/Binarização, dentre outras.


### Tratamento de Dados Perdidos ou Inexistentes (NaN)


Não é incomum que um dado não tenha um ou mais valores de atributos, devido a informações não coletadas ou, até mesmo, esses atributos não se aplicarem às instâncias de dados. Contudo, independente do motivo em que há a falta de dados, é necessário realizar um tratamento para evitar problemas de análise. O tratamento para os chamados "dados perdidos" (ou inexistentes) pode ser realizado de duas principais formas: substituir os valores pela mediana daquele atributo; ou simplesmente descartar aquele dado.


Para verificar se algum dado está faltando, **caso não seja indicado pela descrição do _dataset_**, pode ser realizado a seguinte operação de força-bruta:


In [ ]:
# Substituindo os dados faltantes '?' por 'np.NaN' para ser possível analisar
data = data.replace('?', np.NaN)

print('Número de Instâncias = {0}'.format(data.shape[0]))
print('Número de Atributos = {0}'.format(data.shape[1]))

# Mostrando a quantidade total de dados inválidos, por atributo
print('Número de Dados Perdidos:')
for col in data.columns:
    print('\t{0}: {1}'.format(col, data[col].isna().sum()))


Como é possível ver, não há nenhum dado perdido neste _dataset_ e, desta forma, não é necessário realizar nenhum método de tratamento neste contexto.


### Tratamento de _Outliers_ (Ruídos)


Os _Outliers_ simbolizam dados com características que são consideravelmente diferentes da maioria dos outros dados em um _dataset_. Em outras palavras, simbolizam ruídos que atrapalham ou ajudam na análise dos dados, dependendo do objetivo.

Para identificá-los, é possível por duas abordagens: com base na possibilidade de obter exemplos rotulados pelo usuário, como pelos métodos supervisionados; ou com base em suposições sobre dados normais, como pelo **DBSCAN**. Uma maneira comum consiste em encontrar os percentis e calcular o gráfico de _boxplot_, sendo que os _outliers_ devem seguir as seguintes condições, com base nos limitantes inferior (LB) e superior (UB) do gráfico:

- **Oulier < LB:** Outlier = (Q1 - 1.5 \* IQR), em que Q1 é o percentil de 25%
- **Outlier > UB:** Outlier = (Q3 + 1.5 \* IQR), em que Q3 é o percentil de 75%


Em termos práticos, os gráficos de _boxplot_ deste _dataset_ para os atributos _Area_ e _Convex_Area_, visto com _outlier_ na seção anterior, podem ser vistos a seguir.


In [ ]:
# Plotando o boxplot para 'Area' e 'Convex_Area'
data.boxplot(column=['Area', 'Convex_Area'])


Como pode ser visto acima, ambos os atributos possuem _outliers_ no limitante superior.

Para remover os _outliers_, por meio dos percentis, basta aplicar a seguinte função:


In [ ]:
# Função para remoção dos outliers por meio dos percentis (IQR)
# Disponível em: https://towardsdatascience.com/practical-implementation-of-outlier-detection-in-python-90680453b3ce
def remove_outlier_IQR(df):
    Q1 = df.quantile(0.25)
    Q3 = df.quantile(0.75)
    IQR = Q3-Q1
    LB = Q1-1.5*IQR
    UB = Q3+1.5*IQR
    return df[(df < LB) | (df > UB)]


Aplicando a função e removendo os _outliers_ dos atributos _Area_ e _Convex_Area_:


In [ ]:
# Removendo os outliers de 'Area'
data_outlier_removed = remove_outlier_IQR(data['Area'])

# Atualizando o dataset principal com a remoção dos outliers
data_iqr = data.drop(data_outlier_removed.index)

# Removendo os outliers de 'Convex_Area'
# data_outlier_removed = remove_outlier_IQR(data['Convex_Area'])

# Atualizando o dataset principal com a remoção dos outliers
# data_iqr = data.drop(data_outlier_removed.index)


In [ ]:
# Plotando novamente os boxplots para 'Area' e 'Convex_Area', desta vez com outliers removidos
data_iqr.boxplot(column=['Area', 'Convex_Area'])


### Agregação


A agregação é uma tarefa que consistem em combinar os valores de dois ou mais objetos do _dataset_, de tal forma em que se possa reduzir a dimensionalidade do problema, alterar a granularidade da análise e melhorar a estabilidade dos dados. Deve ser aplicada quando possível, por exemplo em um _dataset_ em que há _as transações de vendas de uma única loja_.

Para exemplificar, a seguir estão os _boxplots_ dos atributos _Area_ e _Convex_Area_ do _dataset_ agregado apenas para a classe **_Cammeo_**.


In [ ]:
# Recuperando apenas os dados relacionados a classe 'Cammeo'
data_cammeo = data[data['Class'] == b'Cammeo']
print(data_cammeo)

# Plotando o boxplot de 'Area' e 'Convex_Area'
data_cammeo.boxplot(column=['Area', 'Convex_Area'])


Nesta agregação, é possível visualizar somente os dados relacionados à classe _Cammeo_ e, desta forma, é possível realizar uma análise mais específica a respeito desta classe, como no que diz respeito a existência de _outliers_ inferiores e superiores dos atributos _Area_ e _Convex_Area_.


### Amostragem


A amostragem, ou _sampling_, é a principal técnica empregada para reduzir dados nos _datasets_ e é utilizada frequentemente para realizar uma investigação preliminar dos dados e a análise final dos mesmos. Ainda que existam vários métodos disponíveis desta técnica, dois são mais recorrentes: amostragem sem substituição, em que cada dado selecionado é removido do conjunto original; e a amostragem com substituição, em que cada dado selecionado não é removido e pode ser selecionado mais de uma vez posteriormente.

O código abaixo exemplifica esta técnica por meio do método de amostragem sem substituição.


In [ ]:
# Copiando o dataset original para exemplificar
data_sampling = cp(data)

# Realizando uma amostragem com 10 dados selecionados aleatoriamente
sample = data_sampling.sample(n=10)
sample


In [ ]:
# Também, é possível realizar a amostragem por meio da seleção percentual de dados desejados
# Seleção de 0,1% dos dados
sample = data_sampling.sample(frac=0.001, random_state=42)
sample


Já abaixo, é possível visualizar a aplicação de amostragem com substituição.


In [ ]:
# Realização de amostragem com substituição e por meio de seleção percentual
sample = data_sampling.sample(frac=0.001, random_state=42, replace=True)
sample


### Normalização e Testes de Normalidade


A normalização é um processo crucial para a análise de dados, uma vez que é responsável por tratar as questões relacionadas com a **magnitude** das características. Em outras palavras, a escala de cada variável influencia diretamente o coeficiente de regressão e, desta forma, as variáveis com uma magnitude mais significativa predominam sobre as que têm um intervalo de magnitude menor. Em termos práticos, quando aplicados em Redes Neurais, essa diferença significativa de magnitude dos atributos afeta negativamente a convergência do gradiente descendente, tornando o processo de treinamento mais lento. Grande parte dos algoritmos de classificação são sensíveis à magnitude, como: Redes Neurais, SVMs, KNN, K-Means, PCA, dentre outros.

Neste cenário, alguns métodos de normalização são bastante utilizados, como o _Standardization_ (_Z-Score_) e o _Normalization_. O _Standardization_ redimensiona a distribuição de valores para que a média dos valores observados seja 0 e o desvio padrão seja 1. Este método preserva a forma da distribuição original e os _outliers_. Já o _Normalization_ subtrai o valor mínimo de todas as variáveis e, em seguida, divide-o pelo intervalo de valores, comprimindo o valor final entre 0 e 1. Neste método, a forma da distribuição original é perdida e os valores estão contidos entre o intervalo [0, 1], sendo bem sensível aos _outliers_.

Para exemplificar, a seguir o _dataset_ é normalizado por meio do _Z-Score_.


In [ ]:
# Mostrando os dados não normalizados
print('Dados Não Normalizados')
print(data.drop(['Class'], axis=1))
print("\n")

# Antes de tudo, remove-se o atributo que define a classe (categórico)
data_normalized = data.drop(['Class'], axis=1)

# Aplicando a Normalização com Z-Score
for column in data_normalized.columns:
    data_normalized[column] = (data_normalized[column] -
                               data_normalized[column].mean()) / data_normalized[column].std()

# Mostrando os dados normalizados
print('Dados Normalizados com Z-Score')
print(data_normalized)


Com isso, é possível traçar os _boxplots_ de todos os atributos de uma maneira mais visível, ao contrário daquele que foi visto anteriormente.


In [ ]:
# Mostrando o gráfico boxplot para todos os atributos
data_normalized.boxplot(figsize=(15, 10))


Também, é possível visualizar se o conjunto suporta o processo de normalização por meio dos chamados Testes de Normalização.
Para tanto, deve-se considerar os seguintes resultados:

- _H0_: A amostra é proveniente de uma população com distribuição normal, com média e desvio-padrão desconhecidos.
- _H1_: A amostra não é proveniente de uma população com distribuição normal.

Os dois principais testes de normalização são: Teste de Shapiro-Wilk e Teste de Kolmogorov-Smirnov. Ambos podem ser acessados por meio da biblioteca _scipy.stats_.


### Seleção de Características


A seleção de características, ou _feature selection_, consistem em um conjunto de técnicas com o objetivo reduzir majoritariamente a dimensionalidade do _dataset_. Essas técnicas são dividas em: _brute-force_, _filter_, _wrapper_ e _embedded_. Também, vale ressaltar a importância deste processo para a obtenção de modelos mais simples e, desta forma, mais fáceis de serem interpretados e treinados por Redes Neurais.

Para exemplificar, a seguir são realizados três métodos de **Filtro de Correlação** da técnica _filter_.


#### Filtro de Correlação


A correlação busca entender, essencialmente, como uma variável se comporta em um cenário onde outra variável está mudando. Ou seja, trata-se de métodos estatísticos para se medir as relações entre as variáveis e busca identificar se existe alguma relação entre elas.

A seguir, são aplicados três tipos de métodos de Filtro de Correlação: de Pearson, de Kendall e de Spearman.


##### Coeficiente de Correlação de Pearson


O Coeficiente de Correlação de Pearson busca encontrar a força das **relações lineares** entre duas variáveis.


In [ ]:
# Mostrando a correlação de Pearson entre os atributos do dataset atual
print("Correlação de Pearson")
data.corr(method='pearson', numeric_only=True)


In [ ]:
# Plotando o mapa de calor da correlação de Pearson
sbn.heatmap(data.corr(method='pearson', numeric_only=True),
            annot=True, cmap='YlGnBu')


##### Coeficiente de Correlação de Kendall


O Coeficiente de Correlação de Kendall busca medir a força da **associação ordinal** entre duas variáveis.


In [ ]:
# Mostrando a correlação de Pearson entre os atributos do dataset atual
print("Correlação de Kendall")
data.corr(method='kendall', numeric_only=True)


In [ ]:
# Plotando o mapa de calor da correlação de Kendall
sbn.heatmap(data.corr(method='kendall', numeric_only=True),
            annot=True, cmap='YlGnBu')


##### Coeficiente de Correlação de Spearman


O Coeficiente de Correlação de Spearman busca encontrar a força das **relações monotônicas (lineares ou não)** entre duas variáveis.


In [ ]:
# Mostrando a correlação de Pearson entre os atributos do dataset atual
print("Correlação de Spearman")
data.corr(method='spearman', numeric_only=True)


In [ ]:
# Plotando o mapa de calor da correlação de Spearman
sbn.heatmap(data.corr(method='spearman', numeric_only=True),
            annot=True, cmap='YlGnBu')


#### _Principal Component Analysis_ (PCA)


O _Principal Component Analysis_ (PCA) é um dos principais métodos para reduzir a dimensionalidade do _dataset_, projetando os dados de seu espaço original de alta dimensão em um espaço de dimensão inferior. Os novos atributos, também chamados de componentes, criados pelo PCA devem ter as seguintes propriedades: são combinações lineares dos atributos originais; são ortogonais entre si; e capturam a quantidade máxima de variação nos dados.

O método do PCA é aplicado a partir dos seguintes passos, de forma ordenada: normalização; computação da matriz de covariância; cálculo dos vetores próprios e os valores próprios da matriz de covariância para identificar os componentes principais; calcular o vetor de características; e reformular os dados ao longo dos eixos de componentes principais.

Para fins exemplares, a aplicação do PCA para este _dataset_ é vista a seguir.


In [ ]:
# Importação do método de normalização Z-Score automático
from sklearn.preprocessing import StandardScaler

# Importação do PCA
from sklearn.decomposition import PCA

# Antes de tudo, remove-se o atributo que define a classe
data_pca = data.drop(['Class'], axis=1)

# Primeiro: normalizar o conjunto de dados
data_pca_normalized = StandardScaler().fit_transform(data_pca)  # Z-Score
print('Dados Normalizados (Z-Score)')
print(data_pca_normalized)


In [ ]:
# Definindo o número de componentes do PCA
n_components = 2  # 2 para colocar em gráfico X por Y

# Aplicando o PCA
pca = PCA(n_components=n_components)
projected_data = pca.fit_transform(data_pca_normalized)

# Mostrando os dados projetados com PCA
print('Dados Projetados com PCA')
print(projected_data)
print("\n")

# Segundo: mostrando a matriz de covariância do PCA
print('Variâncias')
print(pca.explained_variance_ratio_)
print("\n")

# Terceiro: mostrando os componentes do PCA
component_names = ['component {}'.format(
    i) for i in range(len(pca.components_))]
components_pca = pd.DataFrame(
    data=pca.components_, index=component_names, columns=data_pca.columns)
components_pca.head()


In [ ]:
# Gráfico do PCA, com a dimensionalidade reduzida para 2
ins_class = data['Class']
sbn.scatterplot(x=projected_data[:, 0], y=projected_data[:, 1], hue=ins_class)


---


# Conjunto Categórico: **_Car Evaluation_**


**Descrição do Dataset:** este conjunto contém informações sobre a avaliação de 1728 carros por meio dos atributos: preço, manutenção, portas, lugares, espaço no porta-malas e segurança. Além disso, como descrito na publicação, este _dataset_ não possui dados perdidos.

A descrição de cada atributo pode ser vista a seguir.

1. **Preço:** descreve o preço do carro, com base no mercado, e é representado por "alto", "mediano", "baixo" e "muito baixo";
2. **Manutenção:** indica o custo de manutenção do carro, com base no mercado, e é representado por "alto", "mediano", "baixo" e "muito baixo";
3. **Portas:** representa o número de portas do carro, podendo ser "2 portas" ou "4 portas";
4. **Lugares:** descreve a quantidade de passageiros que o carro pode levar, representado por "2 lugares" "4 lugares" e "mais de 4 lugares";
5. **Espaço no Porta-Malas:** indica o tamanho do porta-malas, com valores entre "pequeno", "médio" e "grande";
6. **Segurança:** avalia o nível de segurança do carro, sendo representado por "baixa", "média" e "alta".

Especificamente, os carros podem ser classificados em quatro tipos: "inaceitável", "aceitável", "bom" e "muito bom". Comumente, este conjunto de dados é utilizado para tarefas de classificação, cujo objetivo consiste em prever a classe de avaliação com base nos atributos fornecidos.

Este conjunto de dados pode ser acessado por meio de: [Car Evaluation](https://archive.ics.uci.edu/dataset/19/car+evaluation) (última data de acesso: 15 de set. de 2023).


## <strong> Informações Básicas </strong>


In [ ]:
# Recuperando dados a partir do arquivo e definindo nome dos atributos
data = pd.read_csv('./Datasets/02_00_car.data')
data.columns = ['Buying', 'Maint', 'Doors',
                'Persons', 'Lug_Boot', 'Safety', 'Class']

# Mostra os 5 primeiros e últimos registros
print(data)


In [ ]:
# Mostra os 5 primeiros registros, formatados
data.head()


In [ ]:
# Mostra os 5 últimos registros, formatados
data.tail()


In [ ]:
# Mostra a quantidade de linhas e colunas da tabela (tupla)
data.shape


In [ ]:
# Mostra as informações dos atributos e outras do dataset
data.info()


## <strong> Exploração dos Dados </strong>


### Dados Simples: Média, Desvio-Padrão, Mínimo, Mediana, Máximo


Diferentemente do primeiro _dataset_, este conjunto possui somente atributos categóricos e, portanto, para se calcular as métricas estatísticas acima (e outras) deve-se realizar a conversão dos atributos categóricos para numéricos. Este processo de conversão é denominado de **Encodificação**. Para termos práticos, isso será abordado na seção de **Pré-Processamento**.


### Quantidade de Dados de Cada Classe


In [ ]:
# Retornar a quantidade de classes do dataset por meio da coluna "Class"
# Nota: o nome da coluna deve ser exatamente igual ao do dataset (case-sensitive)
data['Class'].value_counts()


Quando está no contexto de atributos categóricos, uma análise mais recorrente consiste em análise a quantidade de dados (frequência) por atributo, conforme indicado a seguir. Nestas tabelas é indicado a quantidade de classes por atributo, o que indica a recorrência dos mesmos para cada atributo.


In [ ]:
# Tabela de frequência dupla absoluta do atributos categórico 'Buying'
freq_table = pd.crosstab(
    index=data['Buying'], columns=data['Class'], margins=True)
freq_table


No exemplo acima, uma interpretação possível é a seguinte: da classe de carros "aceitos", 108 são de valor "alto" e 115 de valor "médio"; ao passo que da classe "boa" não há nenhum carro de valor "alto", mas há 23 de valor "médio". A partir desta interpretação, pode-se afirmar inicialmente que os carros "aceitos" são, quando comparados aos carros "bons", em maior quantidade com valores "alto" e "médio".

Também, é possível criar a tabela de frequência acima com os percentuais.


In [ ]:
# Tabela de frequência dupla percentual do atributos categórico 'Buying'
freq_table = pd.crosstab(
    index=data['Buying'], columns=data['Class'], margins=True, normalize='all')
freq_table


### Dados Estatísticos Completos Para Cada Atributo


Neste primeiro caso, são incluídos tanto os dados estatísticos básicos, como média e desvio padrão, quanto alguns dos mais avançados, como os percentis (25%, 50% e 75%). Vale ressaltar que, neste caso, há a análise para cada atributo separadamente, ou seja, dados como covariância, a qual relaciona os atributos entre si, não são mostrados.


In [ ]:
# Retornar, para cada atributo (coluna), a descrição estatística completa
# Incluem: média, frequência, mínimo, percentis (25, 50 e 75), dentre outros, quando cabível
data.describe(include='all')


É possível inferir, por exemplo, que a classe "não aceitável" é a mais recorrente com a existência de 1209 registros do total de 1727 deste _dataset_.


## <strong> Pré-Processamento dos Dados </strong>


O pré-processamento consiste na aplicação de diversas técnicas para limpar, selecionar e transformar os dados para melhorar a análise dos mesmos. Algumas técnicas: Agregação, _Sampling_, **_Feature Selection_**, **Redução da Dimensionalidade**, **_Feature Creation_**, Discretização/Binarização, dentre outras.


### Tratamento de Dados Perdidos ou Inexistentes (NaN)


Não é incomum que um dado não tenha um ou mais valores de atributos, devido a informações não coletadas ou, até mesmo, esses atributos não se aplicarem às instâncias de dados. Contudo, independente do motivo em que há a falta de dados, é necessário realizar um tratamento para evitar problemas de análise. O tratamento para os chamados "dados perdidos" (ou inexistentes) pode ser realizado de duas principais formas: substituir os valores pela mediana daquele atributo; ou simplesmente descartar aquele dado.


Para verificar se algum dado está faltando, **caso não seja indicado pela descrição do _dataset_**, pode ser realizado a seguinte operação de força-bruta:


In [ ]:
# Substituindo os dados faltantes '?' por 'np.NaN' para ser possível analisar
data = data.replace('?', np.NaN)

print('Número de Instâncias = {0}'.format(data.shape[0]))
print('Número de Atributos = {0}'.format(data.shape[1]))

# Mostrando a quantidade total de dados inválidos, por atributo
print('Número de Dados Perdidos:')
for col in data.columns:
    print('\t{0}: {1}'.format(col, data[col].isna().sum()))


Como é possível ver, não há nenhum dado perdido neste _dataset_ e, desta forma, não é necessário realizar nenhum método de tratamento neste contexto.


### Conversão de Dados Categóricos para Discretos: Encodificação


Quando trabalha-se com valores categóricos e deseja-se realizar operações estatísticas mais comuns, como média e desvio-padrão, é necessário a conversão destes dados categóricos para numéricos. Além dessas operações básicas, como será visto a seguir, os tratamentos recorrentes no pré-processamento e futuramente na aplicação de Redes Neurais normalmente exigem o uso de dados somente numéricos.

Existem vários métodos para realizar a conversão de dados categóricos para discretos: _One Hot Encoding_, _Dummy Encoding_, _Label Encoding_, _Binary Encoding_, dentre outros. Para fins de exemplificação, a seguir é aplicado o _One Hot Encoding_ para este _dataset_.


In [ ]:
# Utilização do One Hot Encoder para conversão
# from sklearn.preprocessing import OneHotEncoder

# Aplicando o One Hot Encoder em todos os atributos
# data_encoded = pd.DataFrame(OneHotEncoder(handle_unknown='ignore', sparse_output=False).fit_transform(data[cols]))

# Aplicando o One Hot Encoder em todos os atributos
data_encoded = pd.get_dummies(data=data, columns=data.columns)
print(data_encoded)
data_encoded.info()


Como mostra acima, cada atirbuto de cada classe virou um valor binário _True_ ou _False_.


In [ ]:
# Percorrer cada atributo (coluna) e mostrar os dados estatísticos básicos de cada um
print('Média')
print(data_encoded.mean())
print('\n')

print('Desvio-Padrão')
print(data_encoded.std())


In [ ]:
# Gráfico de boxplot para os dados encodificados
data_encoded.boxplot(figsize=(30, 5))


### Amostragem


A amostragem, ou _sampling_, é a principal técnica empregada para reduzir dados nos _datasets_ e é utilizada frequentemente para realizar uma investigação preliminar dos dados e a análise final dos mesmos. Ainda que existam vários métodos disponíveis desta técnica, dois são mais recorrentes: amostragem sem substituição, em que cada dado selecionado é removido do conjunto original; e a amostragem com substituição, em que cada dado selecionado não é removido e pode ser selecionado mais de uma vez posteriormente.

O código abaixo exemplifica esta técnica por meio do método de amostragem sem substituição.


In [ ]:
# Copiando o dataset original para exemplificar
data_sampling = cp(data)

# Realizando uma amostragem com 10 dados selecionados aleatoriamente
sample = data_sampling.sample(n=10)
sample


In [ ]:
# Também, é possível realizar a amostragem por meio da seleção percentual de dados desejados
# Seleção de 1% dos dados
sample = data_sampling.sample(frac=0.01, random_state=42)
sample


Já abaixo, é possível visualizar a aplicação de amostragem com substituição.


In [ ]:
# Realização de amostragem com substituição e por meio de seleção percentual
sample = data_sampling.sample(frac=0.01, random_state=42, replace=True)
sample


### Normalização e Testes de Normalidade


A normalização é um processo crucial para a análise de dados, uma vez que é responsável por tratar as questões relacionadas com a **magnitude** das características. Em outras palavras, a escala de cada variável influencia diretamente o coeficiente de regressão e, desta forma, as variáveis com uma magnitude mais significativa predominam sobre as que têm um intervalo de magnitude menor. Em termos práticos, quando aplicados em Redes Neurais, essa diferença significativa de magnitude dos atributos afeta negativamente a convergência do gradiente descendente, tornando o processo de treinamento mais lento. Grande parte dos algoritmos de classificação são sensíveis à magnitude, como: Redes Neurais, SVMs, KNN, K-Means, PCA, dentre outros.

Neste cenário, alguns métodos de normalização são bastante utilizados, como o _Standardization_ (_Z-Score_) e o _Normalization_. O _Standardization_ redimensiona a distribuição de valores para que a média dos valores observados seja 0 e o desvio padrão seja 1. Este método preserva a forma da distribuição original e os _outliers_. Já o _Normalization_ subtrai o valor mínimo de todas as variáveis e, em seguida, divide-o pelo intervalo de valores, comprimindo o valor final entre 0 e 1. Neste método, a forma da distribuição original é perdida e os valores estão contidos entre o intervalo [0, 1], sendo bem sensível aos _outliers_.

Contudo, para atributos categóricos, este processo foi dado anteriormente com a **Encodificação**, uma vez que transforma os valores categóricos em discretos e, desta forma, sendo possíveis de processamento para os algoritmos de classificação, por exemplo.


### Seleção de Características


A seleção de características, ou _feature selection_, consistem em um conjunto de técnicas com o objetivo reduzir majoritariamente a dimensionalidade do _dataset_. Essas técnicas são dividas em: _brute-force_, _filter_, _wrapper_ e _embedded_. Também, vale ressaltar a importância deste processo para a obtenção de modelos mais simples e, desta forma, mais fáceis de serem interpretados e treinados por Redes Neurais.

Para exemplificar, a seguir são realizados três métodos de **Filtro de Correlação** da técnica _filter_.


#### Filtro de Correlação


A correlação busca entender, essencialmente, como uma variável se comporta em um cenário onde outra variável está mudando. Ou seja, trata-se de métodos estatísticos para se medir as relações entre as variáveis e busca identificar se existe alguma relação entre elas.

A seguir, são aplicados três tipos de métodos de Filtro de Correlação: de Pearson, de Kendall e de Spearman.


##### Coeficiente de Correlação de Pearson


O Coeficiente de Correlação de Pearson busca encontrar a força das **relações lineares** entre duas variáveis.


In [ ]:
# Mostrando a correlação de Pearson entre os atributos do dataset atual
print("Correlação de Pearson")
data_encoded.corr(method='pearson', numeric_only=True)


In [ ]:
# Plotando o mapa de calor da correlação de Pearson
plt.figure(figsize=(35, 10))
sbn.heatmap(data_encoded.corr(method='pearson', numeric_only=True),
            annot=True, cmap='YlGnBu', )


##### Coeficiente de Correlação de Kendall


O Coeficiente de Correlação de Kendall busca medir a força da **associação ordinal** entre duas variáveis.


In [ ]:
# Mostrando a correlação de Pearson entre os atributos do dataset atual
print("Correlação de Kendall")
data_encoded.corr(method='kendall', numeric_only=True)


In [ ]:
# Plotando o mapa de calor da correlação de Kendall
plt.figure(figsize=(35, 10))
sbn.heatmap(data_encoded.corr(method='kendall', numeric_only=True),
            annot=True, cmap='YlGnBu')


##### Coeficiente de Correlação de Spearman


O Coeficiente de Correlação de Spearman busca encontrar a força das **relações monotônicas (lineares ou não)** entre duas variáveis.


In [ ]:
# Mostrando a correlação de Pearson entre os atributos do dataset atual
print("Correlação de Spearman")
data_encoded.corr(method='spearman', numeric_only=True)


In [ ]:
# Plotando o mapa de calor da correlação de Spearman
plt.figure(figsize=(35, 10))
sbn.heatmap(data_encoded.corr(method='spearman', numeric_only=True),
            annot=True, cmap='YlGnBu')


#### _Principal Component Analysis_ (PCA)


O _Principal Component Analysis_ (PCA) é um dos principais métodos para reduzir a dimensionalidade do _dataset_, projetando os dados de seu espaço original de alta dimensão em um espaço de dimensão inferior. Os novos atributos, também chamados de componentes, criados pelo PCA devem ter as seguintes propriedades: são combinações lineares dos atributos originais; são ortogonais entre si; e capturam a quantidade máxima de variação nos dados.

O método do PCA é aplicado a partir dos seguintes passos, de forma ordenada: normalização; computação da matriz de covariância; cálculo dos vetores próprios e os valores próprios da matriz de covariância para identificar os componentes principais; calcular o vetor de características; e reformular os dados ao longo dos eixos de componentes principais.

Contudo, ainda que seja possível utilizar o PCA em dados binários, como no _dataset_ atual aplicado com o _One Hot Encoding_, não é recomendado, tendo em vista que o PCA é designado, majoritariamente, para **variáveis contínuas**.


---


# Conjunto Misto (Numérico e Categórico): **_Abalone_**


**Descrição do Dataset:** este conjunto apresenta um problema de previsão (regressão) da idade de um abalone (espécie de molusco marinho) a partir de medições físicas, as quais são os atributos deste conjunto de dados.

A descrição de cada atributo pode ser vista a seguir.

1. **Sexo (categórico):** descreve o sexo do abalone, podendo ser "M" para masculino, "F" para feminino e "I" para infantil;
2. **Comprimento da Concha (decimal):** representa o comprimento médio da concha, em milímetros;
3. **Diâmetro (decimal):** representa o diâmetro médio da concha, em milímetros;
4. **Altura (decimal):** representa a altura da concha, em milímetros;
5. **Peso Integral (decimal):** representa o peso integral do abalone, incluindo a carne e a concha, em gramas;
6. **Peso da Carne (decimal):** representa o peso da carne do abalone, em gramas;
7. **Peso das Vísceras (decimal):** representa o peso das vísceras (órgãos internos), em gramas;
8. **Peso da Concha (decimal):** representa o peso da concha, em gramas;
9. **Anéis (inteiro):** representa a quantidade de anéis do abalone e, a partir deste atributo, é possível encontrar a idade, em anos, acrescendo com o valor de _1,5_.

Por se tratar de um problema com foco em regressão, há diversas classes (atributo _Anéis_).

Este conjunto de dados pode ser acessado por meio de: [Abalone](https://archive.ics.uci.edu/dataset/1/abalone) (última data de acesso: 15 de set. de 2023).
